In [2]:
!pip install pysrt

     |████████████████████████████████| 104 kB 5.2 MB/s 
  Created wheel for pysrt: filename=pysrt-1.1.2-py3-none-any.whl size=13442 sha256=d7dd63f4619a58c18d20fafda741279e99a3b1ae61aaa51020c00fa9904ad3a2
  Stored in directory: /root/.cache/pip/wheels/b2/f8/e8/a26be4111ab5ec931e845777e574d1483b4adddc50d3e591a6
Successfully built pysrt


In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
import os
os.chdir('/content/drive/MyDrive/Prof_Blas_Reg')
os.getcwd()

## Import Packages

In [ ]:
import json
import re
import pysrt
import pandas as pd

In [ ]:
configJsonPath = "config.json"
category = ["Profanity", "Blasphemy"]


category = [x.lower() for x in category]
with open(configJsonPath, 'r') as f:
    configJson = json.load(f)

In [ ]:
def toLower(dictionary):

    def tryIterate(k):
        return lowerByLevel(k) if isinstance(k, dict) else k

    def tryLower(k):
        return k.lower() if isinstance(k, str) else k

    def lowerByLevel(data):
        return dict((tryLower(k), tryIterate(v)) for k, v in data.items())

    return lowerByLevel(dictionary)

In [ ]:
def context_match(element, text):
    label = element['label']
    context = element['context'].replace('*','\*').replace('$','\$')
#     print(context)
    matchCaseInsensitive = re.compile(context, re.IGNORECASE)
    matched = matchCaseInsensitive.search(text)
#     matches = re.finditer(context, text, re.MULTILINE)
#     for match in matches:
#         print(match)
    if matched:
        return label

In [ ]:
def parser(text):
    matched_labels = []
    matched_class = []
    for cat in range(0,len(configJson)):
        offensive_list = toLower(configJson[cat])
        off_class = list(offensive_list.keys())[0]
        if off_class in category:
            for idx in range(0,len(offensive_list[off_class])):
                element = toLower(offensive_list[off_class][idx])
                if context_match(element, text):
                    matched_labels.append(context_match(element, text))
                    matched_class.append(off_class.title())

    return matched_labels, matched_class
            

In [ ]:
# parser(r"For God's Sake")

In [ ]:
def main(srt_file_path, srt_file_name):
    df = pd.DataFrame()
    subs = pysrt.open(srt_file_path + srt_file_name)
    for sub in subs:
        word_list, pred_list = parser(sub.text)
        # print(sub.text)
        df = df.append({'text': sub.text, 'start':sub.start, 'end': sub.end, 'word': ','.join(word_list), 'pred': ','.join(pred_list)}, ignore_index=True )
        # print()
    df = df.reindex(['text', 'start', 'end', 'word', 'pred'], axis=1)
    return df

## Run

In [ ]:
srt_file_path = r"srt_files/"
srt_file_name = r"Bully 720pED.srt"


output_df = main(srt_file_path, srt_file_name)
file_name = 'output/' + ''.join(srt_file_name.split('.')[:-1]) + ".csv"
output_df.to_csv(file_name,index = False)
output_df

text  ...                 pred
0         I want you\nto suck my big dick.  ...  Profanity,Profanity
1                     Marty honey, dinner.  ...                     
2            I want you\nto lick my balls.  ...            Profanity
3                         <i>His cock?</i>  ...            Profanity
4           <i>His cock was beautiful.</i>  ...            Profanity
..                                     ...  ...                  ...
95                          Fuck you, man.  ...            Profanity
96         I'm sorry, man.\nLet's just go.  ...                     
97                    Let's go, I'm sorry.  ...                     
98  Get your fuckin' hands\noff me, Bobby.  ...            Profanity
99                       I'm really sorry.  ...                     

[100 rows x 5 columns]